In [29]:
import jsonpickle

F,B,E=jsonpickle.decode(open('../ASM_calibration_default-full-tables_FSW_R3.3_AC_DIFF_G5_or_DC_DIFF_G1_R=0-0-0_kappa=1e-04_2021-09-08.json').read())

decl = ""
for prefix,matrix in (("float mag_calibration_matrices_f{freq_index}[NB_BINS_PER_SM * NB_ELEMENTS_MAG_CAL_MATRIX * FLOATS_PER_COMPLEX] = {{",B),
                      ("float elec_calibration_matrices_f{freq_index}[NB_BINS_PER_SM * NB_ELEMENTS_ELEC_CAL_MATRIX * FLOATS_PER_COMPLEX] = {{",E)):
    for freq_index in range(3):
        decl += prefix.format(freq_index=freq_index) +"\n"
        decl += ', '.join(map(lambda item: f"{item.real}f, {item.imag}f", matrix[freq_index].flatten()))
        decl += "};\n\n"

code = f"""
#include "fsw_params_processing.h"

{decl}

"""
with open('../src/processing/calibration_matrices.c', 'w') as src:
    src.write(code)

In [30]:
!clang-format --style=file -i ../src/processing/calibration_matrices.c
!head ../src/processing/calibration_matrices.c


#include "fsw_params_processing.h"

float mag_calibration_matrices_f0[NB_BINS_PER_SM * NB_ELEMENTS_MAG_CAL_MATRIX * FLOATS_PER_COMPLEX]
    = { 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f,
          0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f,
          0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f,
          0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f,
          0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f,
          0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f, 0.0f,
